#Modules

In [4]:
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.model_selection import GridSearchCV


In [5]:
path='C:\\Users\\jyoti\\Desktop\\NLP\\NLP-project-\\'

In [6]:
bow_data=pd.read_csv(path + 'Dataset//bow_dataset.csv')
tf_idf_data=pd.read_csv(path + 'Dataset//tfidf_dataset.csv')
w2v_df=pd.read_csv(path + 'Dataset//w2v_df.csv')
d2v_df=pd.read_csv(path + 'Dataset//d2v_df.csv')

#bow

In [7]:
bow_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,861,862,863,864,865,866,867,869,871,874
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0.632390,0.282576,8,0,10
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0.405556,0.168056,9,0,2
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0.000000,0.000000,25,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.333333,0.250000,28,0,0
4,4,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0.333333,0.166667,28,2,0


In [8]:
dropped = ['Unnamed: 0']
data = bow_data.drop(dropped,axis=1)
bow_data=data

In [9]:
# y = np.ravel(data[806])
y=bow_data.iloc[ :, -1:]  #Target Variable          
X=bow_data.iloc[: , :-1]  #Feature Matrix 

# y.head()

In [10]:
#converted the dataframe

X=pd.DataFrame(X)
y=pd.DataFrame(y)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

In [12]:
def hyperparameter_Tuning():
    model = xgb.XGBRegressor()
    parameters = {'nthread':[4],
                  'objective':['reg:linear'],
                  'learning_rate': [.03, 0.05, .07], #so called `eta` value
                  'max_depth': [5, 6, 7],
                  'min_child_weight': [4],
                  'silent': [1],
                  'subsample': [0.7],
                  'colsample_bytree': [0.7],
                  'n_estimators': [500]}

    model_grid = GridSearchCV(model,parameters,cv = 2,n_jobs = 5, verbose=True)

    model_grid.fit(x_train, y_train)
    model_grid.best_score_
    model_grid.best_params_
    return model_grid.best_params_

In [13]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.03,max_depth = 5, min_child_weight=4,nthread = 4, n_estimators = 20)
xg_reg.fit(x_train, y_train)
y_pred = xg_reg.predict(x_test)


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
R2=r2_score(y_test,y_pred)
print("R2 Score: %f" % (R2))
print()        

[18:56:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 2.487375
R2 Score: 0.249058



In [14]:
y_test

y_pred=pd.DataFrame(y_pred)
y_pred

,0
0,0.804422
1,0.804422
2,0.804422
3,0.804422
4,1.160828
...,...
8344,0.804422
8345,1.178230
8346,0.804422
8347,0.980845


In [15]:
def plot_feature_importance(importance,names,model_type):

#Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=(20,20))
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + 'FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

# plot_feature_importance(xg_reg.feature_importances_,x_train.columns,'XG BOOST')

#tfid

In [16]:
tf_idf_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,815,816,817,818,819,820,821,823,825,828
0,0,0.0,0.0,0.0,0.0,0.202957,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.632390,0.282576,8,0,10
1,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0.405556,0.168056,9,0,2
2,2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0.000000,0.000000,25,0,0
3,3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.333333,0.250000,28,0,0
4,4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0.333333,0.166667,28,2,0


In [17]:
y=tf_idf_data.iloc[ :, -1:]  #Target Variable          
X=tf_idf_data.iloc[: , :-1]  #Feature Matrix 

# y.head()

In [18]:
#converted the dataframe

X=pd.DataFrame(X)
y=pd.DataFrame(y)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

In [20]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.03,max_depth = 5, min_child_weight=4,nthread = 4, n_estimators = 20)
xg_reg.fit(x_train, y_train)
y_pred = xg_reg.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
R2=r2_score(y_test,y_pred)
print("R2 Score: %f" % (R2))
print()        

[18:57:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 2.449713
R2 Score: 0.271626



In [21]:
y_test

y_pred=pd.DataFrame(y_pred)
y_pred

,0
0,0.783116
1,0.783116
2,0.783116
3,0.783116
4,1.168612
...,...
8344,0.783116
8345,0.920026
8346,0.783116
8347,1.209221


#w2v

In [22]:
y=w2v_df.iloc[ :, -1:]  #Target Variable          
X=w2v_df.iloc[: , :-1]  #Feature Matrix 

X=pd.DataFrame(X)
y=pd.DataFrame(y)
for i in X.columns:
  X[i]=pd.to_numeric(X[i])
for i in y.columns:
  y[i]=pd.to_numeric(y[i])


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.03,max_depth = 5, min_child_weight=4,nthread = 4, n_estimators = 500,silent=1,subsample=0.7)
xg_reg.fit(x_train, y_train)
y_pred = xg_reg.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

R2=r2_score(y_test,y_pred)
print("R2 Score: %f" % (R2))


[18:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


RMSE: 0.015908
R2 Score: 0.941253


#d2v

In [26]:
y=d2v_df.iloc[ :, -1:]  #Target Variable          
X=d2v_df.iloc[: , :-1]  #Feature Matrix

X=pd.DataFrame(X)
y=pd.DataFrame(y)

for i in X.columns:
  X[i]=pd.to_numeric(X[i])
for i in y.columns:
  y[i]=pd.to_numeric(y[i])


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [25]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.03,max_depth = 5, min_child_weight=4,nthread = 4, n_estimators = 500,silent=1,subsample=0.7)
xg_reg.fit(x_train, y_train)
y_pred = xg_reg.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

R2=r2_score(y_test,y_pred)
print("R2 Score: %f" % (R2))

[19:11:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[19:11:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


RMSE: 0.017557
R2 Score: 0.926415
